target format is JSON
{
"demoset":{"bottle":"pullo","station":"asema","shelf":"hylly","bar":"baari"},
"demoset2":{"bottle2":"pullo2","station2":"asema2","shelf2":"hylly2","bar2":"baari2"}
 }

In [70]:
import pandas as pd
import json

In [7]:
nouns = pd.read_excel("./core words completed.xlsx", 0 )
verbs = pd.read_excel("./core words completed.xlsx", 2)
adj = pd.read_excel("./core words completed.xlsx", 1 )
rankings = pd.read_csv("./rankedvocab.csv")

In [8]:
nouns["POS1"] = "n"
nouns.head()
nouns.shape

(2459, 3)

In [9]:
verbs.head()
verbs.columns = ["Finnish","English"]
verbs["POS1"] = "v"
verbs.shape

(1138, 3)

In [10]:
adj.head()
adj["POS1"] = "j"
adj.shape

(569, 3)

In [11]:
rankings.head()
rankings["POS"].value_counts()

POS
n       2688
v       1042
j        692
r        214
fw       167
u         99
m         84
K         62
abbr      14
Name: count, dtype: int64

In [12]:
rankings[rankings["POS"]=="r"].shape

(214, 8)

In [13]:
rankingreduced = rankings[["word","POS","ranking"]]
# rankingreduced = rankings[["word","ranking"]]

rankingreduced.head()
rankingreduced.shape

(5063, 3)

In [14]:
stage1 = pd.merge(rankingreduced,
                  adj,
                  left_on = ["word",'POS'],
                  right_on= ["English",'POS1'],
                  how = "right",
                  left_index = False,
                  right_index = False)
stage1.shape

(569, 6)

In [15]:
stage1.head()

,word,POS,ranking,Finnish,English,POS1
0,able,j,609.0,kykenevä,able,j
1,NaN,NaN,NaN,ylinen,above,j
2,absolute,j,3511.0,absoluuttinen,absolute,j
3,absurd,j,4740.0,absurdi,absurd,j
4,acceptable,j,4477.0,hyväksyttävä,acceptable,j


In [16]:
stage1[stage1["POS1"]=="j"].shape

(569, 6)

In [23]:
stage1["Finnish"].isna().sum()

0

In [29]:
s1clean = stage1[(stage1["Finnish"].notnull()) &((stage1["POS1"]=="j") | (stage1["POS1"]=="r" )|(stage1["POS1"]=="fw"))].copy()
# s1clean = stage1[(stage1["Finnish"].notnull())].copy()

# s1clean["POS"] = "rj" #adjs
# # stage1[(stage1["Finnish"].notnull())&(stage1["POS"]=="j")].shape #adjs
s1clean.shape

(569, 6)

In [31]:
s1clean.head()

,word,POS,ranking,Finnish,English,POS1
0,able,j,609.0,kykenevä,able,j
1,NaN,NaN,NaN,ylinen,above,j
2,absolute,j,3511.0,absoluuttinen,absolute,j
3,absurd,j,4740.0,absurdi,absurd,j
4,acceptable,j,4477.0,hyväksyttävä,acceptable,j


In [33]:
stage2 = pd.merge(rankingreduced, 
                  nouns, 
                  left_on = ["word",'POS'],
                  right_on= ["English",'POS1'],
                  how = "right",
                  left_index = False, 
                  right_index = False)

In [35]:
s2clean = stage2[(stage2["Finnish"].notnull()) & (stage2["POS1"]=="n") ] #nouns
s2clean.shape

(2459, 6)

In [37]:
stage3 = pd.merge(rankingreduced,
                  verbs,
                  left_on = ["word",'POS'],
                  right_on= ["English",'POS1'],
                  how = "right",
                  left_index = False,
                  right_index = False)

In [39]:
s3clean = stage3[stage3["Finnish"].notnull()]
s3clean.shape
#verbs

(1138, 6)

In [41]:
#adj + v + n
569+1138+2459


4166

In [43]:
total = pd.concat([s1clean,s2clean,s3clean], ignore_index = True)

In [46]:
total[(total["ranking"]>= 0) & (total["ranking"]<5000)]["POS"].value_counts()

POS
n    2036
v     749
j     470
Name: count, dtype: int64

In [48]:
# now split by ranking 
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="v")].shape

(100, 6)

In [50]:
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="n")].shape

(111, 6)

In [52]:
total[ ((total["ranking"]>= 0) & (total["ranking"]<500)) & (total['POS']=="fw")].shape


(0, 6)

In [54]:
total.to_csv("test.csv")

In [56]:
total = total.sort_values(by = 'ranking')#
total.reset_index(inplace=True, drop=True)

In [84]:
total.tail()
total.value_counts("POS")

POS
n    2058
v     755
j     482
Name: count, dtype: int64

In [60]:
# bounds = [0,500,1000,1500,2000,2500,3000,3500,4000]
# lower = bounds[0]
# upper = bounds[1]

# POS = ["v","n","rj"]

# batch = [ total[((total["ranking"]>= lower) & (total["ranking"]<upper)) & (total['POS']== pos)] for pos in POS]
# pd.concat(batch).shape
# name = f"{lower}-{higher}"
   
# # def setbuilder(lower,upper):
# #     [total[ ((total["ranking"]> lower) & (total["ranking"]<upper)) & (total['POS']== pos)].shape for pos in POS

In [62]:
# def make_set(start,end,set_name, datasource):
#     json_extract = f'"{set_name}":{{\n'
#     last_val = ''
#     last_entry = ''
#     for index, row in datasource[start:end].iterrows():
#         # print(last_val)
#         try:
#             if last_val == row['English']:
#                 # print(f"found a duplicate of {row['English']}" )
#                 # print(last_val, row["English"])
#                  # Modify last_entry to include both values
#                 json_extract = json_extract.rstrip('", \n')  # Remove the trailing comma and space
#                 json_extract += f', {row["Finnish"]}", \n'  # Add the Finnish value from the duplicate 
                
#                 if index-start < end-start-1:
#                     new_entry+= ", \n"
#                     # remember previous entry for nest round
#                     last_val = row['English']
#                 else:
#                     new_entry+="\n"
#                     # clear memory for next stage start
#                     last_val = ''
#             else:
#                 new_entry = f'''\t"{row['English']} ({row['POS1']})":"{row['Finnish']}"'''
                
#                 if index-start < end-start-1:
#                     new_entry+= ", \n"
#                     # remember previous entry for nest round
#                     last_val = row['English']
#                 else:
#                     new_entry+="\n"
#                     # clear memory for next stage start
#                     last_val = ''
                    
#                 json_extract += new_entry
#         except:
#             print("No data found ... looks like the end of the data")
#     json_extract+="\t}, \n"
   
#     return json_extract

In [64]:
# with open("vitalvocab.txt", "w") as j:
#     j.write('{\nword_sets: { \n')
#     for i in range(0,4300,40):
#         j.write(make_set(i,i+40,f"word_set{int(i/40)}",total))

#     j.write('\n} \n})

In [66]:
# total.to_csv("./check me.csv", encoding = "utf-8-sig")

In [86]:
POS_MAP = {
    "V": "verb",
    "N": "noun",
    "J":"adjective"
}

def make_set(start, end, set_name, datasource):
    word_list = []

    for index, row in datasource[start:end].iterrows():
        try:
            pos_mapped = POS_MAP.get(row["POS1"].strip().upper(), "other")

            word_entry = {
                "word": row["English"],
                "partOfSpeech": pos_mapped,
                "translation": row["Finnish"]
            }

            # Check for duplicate English words and merge translations
            existing = next((item for item in word_list if item["word"] == row["English"]), None)
            if existing:
                existing["translation"] += f", {row['Finnish']}"
            else:
                word_list.append(word_entry)

        except Exception as e:
            print(f"Skipping row at index {index} due to error: {e}")

    return f'"{set_name}": {json.dumps(word_list, indent=4, ensure_ascii=False)},\n'


with open("vitalvocab.txt", "w", encoding="utf-8") as j:
    j.write('{\nword_sets: { \n')
    for i in range(0,4300,40):
        j.write(make_set(i,i+40,f"word_set{int(i/40)}",total))

    j.write('\n} \n}')